In [5]:

from mytorch import Model
from mytorch.operations import Flatten
from mytorch.activations import ReLU, Softmax
from mytorch.layers import Linear, Dropout


import keras
class TwoLayerModel(Model):
    def __init__(self):
        super().__init__()
        self.flatten = Flatten()
        self.linear1 = Linear(784, 128)
        self.linear2 = Linear(128, 10)
        self.dropout = Dropout(0.3)
        self.relu = ReLU()
        self.softmax = Softmax(dim=1)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x
    

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [6]:
from mytorch.optimizers import SGD
from mytorch.losses import MSE
from mytorch import Tensor
import numpy as np

model = TwoLayerModel()
loss_fn = MSE()
optimizer = SGD(model.get_parameters(), lr=0.001)

batch_size = 128
epochs = 20

for epoch in range(epochs):
    model.train()
    for i in range(0, len(x_train), batch_size):
        batch_x = x_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        batch_x = Tensor(batch_x)
        batch_y = Tensor(batch_y)
        
        output = model(batch_x)
        
        loss = loss_fn(output, batch_y)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
    model.eval()
    output = model(x_test)
    acc = np.sum(np.argmax(output.data, axis=1) == np.argmax(y_test, axis=1)) / len(x_test)
    print("Epoch: %d, Loss: %.5f, Accuracy: %.5f" % (epoch+1, loss.data, acc))

Epoch: 1, Loss: 0.06232, Accuracy: 0.90570
Epoch: 2, Loss: 0.07028, Accuracy: 0.91930
Epoch: 3, Loss: 0.06384, Accuracy: 0.93030
Epoch: 4, Loss: 0.06592, Accuracy: 0.93840
Epoch: 5, Loss: 0.06692, Accuracy: 0.94270
Epoch: 6, Loss: 0.06216, Accuracy: 0.94540
Epoch: 7, Loss: 0.06389, Accuracy: 0.94740
Epoch: 8, Loss: 0.06218, Accuracy: 0.95040
Epoch: 9, Loss: 0.06157, Accuracy: 0.95310
Epoch: 10, Loss: 0.06484, Accuracy: 0.95620
Epoch: 11, Loss: 0.06454, Accuracy: 0.95660
Epoch: 12, Loss: 0.05964, Accuracy: 0.95820
Epoch: 13, Loss: 0.06346, Accuracy: 0.95790
Epoch: 14, Loss: 0.06106, Accuracy: 0.95840
Epoch: 15, Loss: 0.06693, Accuracy: 0.96070
Epoch: 16, Loss: 0.06523, Accuracy: 0.96200
Epoch: 17, Loss: 0.06439, Accuracy: 0.96270
Epoch: 18, Loss: 0.06469, Accuracy: 0.96350
Epoch: 19, Loss: 0.05649, Accuracy: 0.96440
Epoch: 20, Loss: 0.06379, Accuracy: 0.96490


In [7]:
model.save('model.npz')

In [8]:
model = TwoLayerModel()
model.load('model.npz')

model.eval()
output = model(x_test)
acc = np.sum(np.argmax(output.data, axis=1) == np.argmax(y_test, axis=1)) / len(x_test)
print("Test Accuracy: %.5f" % (acc))

Test Accuracy: 0.96490
